# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [1]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("https://jportal.mdcourts.gov/license/index_disclaimer.jsp")


**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [2]:
search_button = driver.find_element_by_xpath('//*[@id="checkbox"]')
search_button.click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [3]:
from selenium.webdriver.common.keys import Keys
element = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
element.send_keys(Keys.RETURN)

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [4]:
search_license = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]')
search_license.click()

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [5]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_name('slcJurisdiction'))
#select = Select(seclect_tag) sent an elemnt to be considered a selection
select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [6]:
trade_input = driver.find_element_by_name('txtTradeName')
trade_input.send_keys("Vap%")


### How do you click the submit button or submit the form?

In [7]:
info = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
info.click()

### How can you find and click the 'Next' button on the search results page?

In [8]:
#next = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
#next.click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [9]:
from bs4 import BeautifulSoup
import requests


In [10]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.
#business_headers = doc.find_all('tr',class_='searchfieldtitle')
#len(business_headers)

In [11]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.
#for header in business_headers:
 #   rows = header.find_next_siblings('tr')
 #   print("HEADER is", header.text.strip())
 #   print("ROW 0 IS", rows[0].text.strip())
 #   print("ROW 1 IS", rows[1].text.strip())
 #   print("ROW 2 IS", rows[2].text.strip())
 #   print("ROW 3 IS", rows[3].text.strip())
 #   print("----")

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [12]:
doc = BeautifulSoup(driver.page_source, 'html.parser')
doc

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="EN" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>Maryland Judiciary Business Licenses Online</title>
<link href="theme/styles.css" rel="STYLESHEET" type="text/css"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
</head>
<body>
<!-- HEADER AREA - Header graphic, company logo and link images go here-->
<table border="0" cellpadding="0" cellspacing="0" summary="Page Layout Table" width="100%">
<tbody><tr>
<td colspan="3">
<img alt="MARYLAND BUSINESS LICENSES ONLINE" height="66" src="images/header_new.gif" width="596"/><img alt="" src="images/spacer.gif" width="35"/>
</td>
</tr>
<tr>
<td class="headerline">
<img alt="blank spacer" height="18" src="images/spacer.gif" width="35"/>
</td>
<td align="LEFT" class="headerline">
<table border="0" cellpadding="0" cellspacing="0" summary="Navigation Menu">
<tbody><tr valign="top">
<td>
<img alt="" height="18

In [13]:
business_headers = doc.find_all('tr',class_='searchfieldtitle')
len(business_headers)

5

In [14]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.
mylist = []
for header in business_headers:
    rows = header.find_next_siblings('tr')
    print("HEADER is", header.span.text.strip())
    print("Name is", rows[0].find_all('td')[1].text)
    if (rows[0].find_all('td')[2].span):
        print(rows[0].find_all('td')[2].span.text, rows[0].find_all('td')[2].span.next_sibling)
    print("Address 1", rows[1].find_all('td')[1].text)
    if (rows[1].find_all('td')[2].span):
        print(rows[1].find_all('td')[2].span.text, rows[1].find_all('td')[2].span.next_sibling)
    print("Addreaa 2", rows[2].find_all('td')[1].text)
    if (rows[2].find_all('td')[2].span):
        print(rows[2].find_all('td')[2].span.text, rows[2].find_all('td')[2].span.next_sibling)
    print("Address 3", rows[3].text.strip())
    
    dic = {}
    dic['HEADER'] = header.span.text.strip()
    dic['name'] = rows[0].find_all('td')[1].text
    dic['Address 1'] = rows[1].find_all('td')[1].text
    dic['Address 2'] = rows[2].find_all('td')[1].text
    dic['Address 3'] = rows[3].text.strip()
    if (rows[0].find_all('td')[2].span):
        lic = rows[0].find_all('td')[2].span.text
        status = rows[0].find_all('td')[2].span.next_sibling
        dic[lic] = status
    else:
        dic['Lic. Status:'] = ''
    if(rows[1].find_all('td')[2].span):
        License = rows[1].find_all('td')[2].span.text
        number = rows[1].find_all('td')[2].span.next_sibling
        dic[License] = number
    else:
        dic['License:'] = ''
    if(rows[2].find_all('td')[2].span):
        issued = rows[2].find_all('td')[2].span.text
        date = rows[2].find_all('td')[2].span.next_sibling
        dic[issued] = date
    else:
        dic['Issued Date:'] = ''
    
    mylist.append(dic)
    print("------------------------------------------")
    

HEADER is VAPE IT STORE I
Name is AMIN NARGIS
Lic. Status:  Issued
Address 1 1724 N SALISBURY BLVD UNIT 2
License:  22173807
Addreaa 2 SALISBURY, MD 21801
Issued Date:  4/27/2017
Address 3 Wicomico County
------------------------------------------
HEADER is VAPE IT STORE II
Name is AMIN NARGIS
Lic. Status:  Issued
Address 1 1015 S SALISBURY BLVD
License:  22173808
Addreaa 2 SALISBURY, MD 21801
Issued Date:  4/27/2017
Address 3 Wicomico County
------------------------------------------
HEADER is VAPEPAD THE
Name is ANJ DISTRIBUTIONS LLC
Lic. Status:  Issued
Address 1 2299 JOHNS HOPKINS ROAD
License:  02104436
Addreaa 2 GAMBRILLS, MD 21054
Issued Date:  4/05/2017
Address 3 Anne Arundel County
------------------------------------------
HEADER is VAPE FROG
Name is COX TRADING COMPANY L L C
Lic. Status:  Issued
Address 1 110 S. PINEY RD
License:  17165957
Addreaa 2 CHESTER, MD 21619
Issued Date:  5/31/2017
Address 3 Queen Anne's County
------------------------------------------
HEADER is VA

In [15]:
mylist

[{'Address 1': '1724 N SALISBURY BLVD UNIT 2',
  'Address 2': 'SALISBURY, MD 21801',
  'Address 3': 'Wicomico County',
  'HEADER': 'VAPE IT STORE I',
  'Issued Date:': ' 4/27/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 22173807',
  'name': 'AMIN NARGIS'},
 {'Address 1': '1015 S SALISBURY BLVD',
  'Address 2': 'SALISBURY, MD 21801',
  'Address 3': 'Wicomico County',
  'HEADER': 'VAPE IT STORE II',
  'Issued Date:': ' 4/27/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 22173808',
  'name': 'AMIN NARGIS'},
 {'Address 1': '2299 JOHNS HOPKINS ROAD',
  'Address 2': 'GAMBRILLS, MD 21054',
  'Address 3': 'Anne Arundel County',
  'HEADER': 'VAPEPAD THE',
  'Issued Date:': ' 4/05/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 02104436',
  'name': 'ANJ DISTRIBUTIONS LLC'},
 {'Address 1': '110 S. PINEY RD',
  'Address 2': 'CHESTER, MD 21619',
  'Address 3': "Queen Anne's County",
  'HEADER': 'VAPE FROG',
  'Issued Date:': ' 5/31/2017',
  'Lic. Status:': ' Issued',
  'License:': '

### Save these into `vape-results.csv`

In [16]:
import pandas as pd

df = pd.DataFrame(mylist)
df.to_csv('Maryland_Business_Licenses_page1.csv', index = False)

### Open `vape-results.csv` to make sure there aren't any extra weird columns

## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.

In [17]:
vape_results = []
while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    business_headers = doc.find_all('tr',class_='searchfieldtitle')
    
    for header in business_headers:
        dic = {}
        rows = header.find_next_siblings('tr')
        dic['HEADER'] = header.span.text.strip()
        dic['name'] = rows[0].find_all('td')[1].text
        dic['Address 1'] = rows[1].find_all('td')[1].text
        dic['Address 2'] = rows[2].find_all('td')[1].text
        dic['Address 3'] = rows[3].text.strip()
        if (rows[0].find_all('td')[2].span):
            lic = rows[0].find_all('td')[2].span.text
            status = rows[0].find_all('td')[2].span.next_sibling
            dic[lic] = status
        else:
            dic['Lic. Status:'] = ''
        if(rows[1].find_all('td')[2].span):
            License = rows[1].find_all('td')[2].span.text
            number = rows[1].find_all('td')[2].span.next_sibling
            dic[License] = number
        else:
            dic['License:'] = ''
        if(rows[2].find_all('td')[2].span):
            issued = rows[2].find_all('td')[2].span.text
            date = rows[2].find_all('td')[2].span.next_sibling
            dic[issued] = date
        else:
            dic['Issued Date:'] = ''
    
        vape_results.append(dic)
        
    try:
        next = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
        next.click()
        
    except:
        break
    

In [18]:
vape_results

[{'Address 1': '1724 N SALISBURY BLVD UNIT 2',
  'Address 2': 'SALISBURY, MD 21801',
  'Address 3': 'Wicomico County',
  'HEADER': 'VAPE IT STORE I',
  'Issued Date:': ' 4/27/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 22173807',
  'name': 'AMIN NARGIS'},
 {'Address 1': '1015 S SALISBURY BLVD',
  'Address 2': 'SALISBURY, MD 21801',
  'Address 3': 'Wicomico County',
  'HEADER': 'VAPE IT STORE II',
  'Issued Date:': ' 4/27/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 22173808',
  'name': 'AMIN NARGIS'},
 {'Address 1': '2299 JOHNS HOPKINS ROAD',
  'Address 2': 'GAMBRILLS, MD 21054',
  'Address 3': 'Anne Arundel County',
  'HEADER': 'VAPEPAD THE',
  'Issued Date:': ' 4/05/2017',
  'Lic. Status:': ' Issued',
  'License:': ' 02104436',
  'name': 'ANJ DISTRIBUTIONS LLC'},
 {'Address 1': '110 S. PINEY RD',
  'Address 2': 'CHESTER, MD 21619',
  'Address 3': "Queen Anne's County",
  'HEADER': 'VAPE FROG',
  'Issued Date:': ' 5/31/2017',
  'Lic. Status:': ' Issued',
  'License:': '

In [19]:
df2 = pd.DataFrame(vape_results)
df2.to_csv('vapes_result.csv', index = False)